In [1]:
import numpy as np
from utils import get_data, get_table, get_predictions, other_stats, add_intervals_to_test
from plots import plot_rmse, plot_finish_groups, plot_interval_checks, plot_finish_age_gender
np.random.seed(2025)

size1, size2 = 250, 4000
save_val = True
train_yr, test_yr = [2021, 2022, 2023], [2024]
train_bos, test_bos = get_data(racename="bos", size_train=size1, size_test=size2, train_lis=train_yr, test_lis=test_yr, save=save_val)
train_nyc, test_nyc = get_data(racename="nyc", size_train=size1, size_test=size2, train_lis=train_yr, test_lis=test_yr, save=save_val)
train_chi, test_chi = get_data(racename="chi", size_train=size1, size_test=size2, train_lis=train_yr, test_lis=test_yr, save=save_val)
data = {"bos": (train_bos, test_bos), "nyc": (train_nyc, test_nyc), "chi": (train_chi, test_chi)}
# test_nyc = pd.read_csv("processed_data/test_nyc.csv")

In [2]:
race = "bos"
test = data[race][1]

model_info = [
    ("M1", f"stan_results/model1/params_{race}.csv", ["alpha", "total_pace"]),
    ("M2", f"stan_results/model2/params_{race}.csv", ["alpha", "total_pace", "curr_pace"]),
    ("M3", f"stan_results/model3/params_{race}.csv", ["alpha", "total_pace", "curr_pace", "male", "age"]),
]
mpreds = {name: get_predictions(test, path, feats_lis=feats, full=False) for (name, path, feats) in model_info}
models, baseline = ["M1", "M2", "M3"], "BL"
test2 = get_table(test, mpreds, baseline_name=baseline)
test2

,id,dist,curr_pace,total_pace,finish,age,gender,year,prop,propleft,male,propxcurr,malexage,alpha,lvl,BL,M1,M2,M3
0,74170,5K,3.720238,3.720238,3.401725,46,M,2024,0.118497,0.881503,1,0.440839,46,1,1,-17.699733,-8.480271,-8.493979,-7.520927
1,81272,5K,3.134796,3.134796,2.956903,69,M,2024,0.118497,0.881503,1,0.371465,69,1,1,-13.496583,2.485325,2.505329,7.585249
2,69740,5K,3.958828,3.958828,3.752001,47,M,2024,0.118497,0.881503,1,0.469111,47,1,1,-9.792383,-2.381137,-2.402780,-1.391869
3,72944,5K,4.022526,4.022526,3.480861,65,M,2024,0.118497,0.881503,1,0.476659,65,1,1,-27.205383,-20.213330,-20.236724,-17.501072
4,80672,5K,3.322259,3.322259,2.998508,42,F,2024,0.118497,0.881503,0,0.393679,0,1,1,-22.855083,-9.503655,-9.497387,-11.070455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,71982,40K,3.477051,3.557769,3.549676,33,F,2024,0.947980,0.052020,0,3.296174,0,1,8,-0.450673,-0.060497,-0.182715,-0.278188
31996,84704,40K,2.423655,2.700695,2.676668,68,M,2024,0.947980,0.052020,1,2.297575,68,1,8,-2.337440,-1.730534,-1.386486,-0.991975
31997,71692,40K,3.293808,3.570154,3.570703,28,F,2024,0.947980,0.052020,0,3.122463,0,1,8,0.030325,0.418379,0.766549,0.605526
31998,79682,40K,3.092146,3.061146,3.061158,64,F,2024,0.947980,0.052020,0,2.931291,0,1,8,0.000860,0.496231,-0.148676,0.190492


In [3]:
tbl = plot_rmse(test2, models, baseline, save_name=race, bar=True)
other_stats(test2[[baseline] + models], test2["finish"], save_name=race)
# test2["model1"].argmax()

File saved: analysis/plots/bos_rmse_bar.png
File saved: analysis/tables/bos_rmse.csv
File saved: analysis/tables/bos_rmse2.csv


,BL,M1,M2,M3,pcnt_BL,pcnt_M1,pcnt_M2,pcnt_M3
Overall RMSE,21.939136,15.043131,14.255973,14.186029,-,-,-,-
Overall R-squared,0.791366,0.901911,0.911907,0.912770,-,-,-,-


In [4]:
c_model = "M2"
a = plot_finish_groups(test2, model=c_model, baseline=baseline, num=4, overall=True, save_name=race, palette="inferno")
plot_finish_age_gender(test2, model=c_model, baseline=baseline, num=4, overall=True, save_name=race, palette="crest", grouping="age")

File saved: analysis/plots/bos_rmse_groups.png
File saved: analysis/plots/bos_rmse_gender_age.png


In [5]:
# mpreds2 = {name: (42195 / 60) / get_predictions(test, path, feats_lis=feats, full=True) for (name, path, feats) in model_info}
# intervals_tbl = add_intervals_to_test(test2, mpreds2, models)
# i_check, i_sizes = plot_interval_checks(intervals_tbl, models, save_name=race)